In [1]:
import os
from os.path import join
import re
from tqdm import tqdm
from file_function import write_log_csv_upload
from PIL import Image, ImageSequence
# PyPDF : pdf를 대상으로 (pdf > img는 되나, img > pdf는 안 됨)
from PyPDF2 import PdfFileReader, PdfFileWriter

path_success_log = r'D:\0.전산\1.진행중과업\파일서버개편\구스캔폴더 작업log\success'

### pillow 여러 폴더 작업

In [17]:
#### csv읽어서 재작업
# import pandas as pd
# df = pd.read_csv(r"D:\업로드완료\광주 모음 대출원장 티와이1\20221117 175127_319_파일병합 실패.csv", header=None)
# df.columns = ["root", "dir", "1"]
# df.dir = df.dir.astype("str")
# dir_list = list(df.dir)
# path = r"D:\업로드완료\광주 모음 대출원장 티와이1"
# print(dir_list[0], type(dir_list[0]))
# for d in dir_list :
#     temp = [x.name for x in os.scandir(join(path, d)) if x.is_file()]
#     if len(temp) != 0 :
#         print(d)

20407804 <class 'str'>


##### 전처리 

In [6]:
# pdf 파일 이름 바꿔 별도로 옮기기
############################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\25.퀀텀_솔림헬프\5.매각사측 수령 스캔본\매각사측 원인서류 스캔본"
addInfo = "서울 채권별 25.퀀텀 5.매각사 원인"
dst_dir = join(r"D:\업로드완료", addInfo, "pdf파일만")
############################################
if not os.path.exists(dst_dir) : os.makedirs(dst_dir)

success, fail = [], []
p_extension = re.compile('pdf$', re.I)
for r, ds, fs in tqdm(os.walk(path)) :
    for f in fs :
        if p_extension.search(f) :
            try :
                stem, ext = os.path.splitext(f)
                new_stem = os.path.split(r)[1] + " " + stem
                new_name = new_stem + ext
                cnt = 0
                while os.path.exists(join(dst_dir, new_name)) :
                    cnt += 1
                    new_name = new_stem + " (" + str(cnt) + ")" + ext

                os.rename(join(r, f), join(dst_dir, new_name))
                success.append([r, f, dst_dir, new_name])
            except Exception as e :
                fail.append([r, f, e.__class__, e])
                continue
write_log_csv_upload(success, path_success_log, addInfo + "pdf파일만")
write_log_csv_upload(fail, path, "pdf파일만-이동 실패")

log 파일이 비어있습니다. path : D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\25.퀀텀_솔림헬프\5.매각사측 수령 스캔본\매각사측 원인서류 스캔본


In [3]:
# 이미지 아닌 파일 있는지 확인
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\25.퀀텀_솔림헬프\5.매각사측 수령 스캔본\매각사측 원인서류 스캔본"
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
for r, ds, fs in os.walk(path) :
    for f in fs :
        if p_extension.search(f) == None :
            print(r, f)

##### 일반(walk, 폴더내의 여러 img파일을 하나의 pdf만들기)

In [9]:
############################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\25.퀀텀_솔림헬프\5.매각사측 수령 스캔본\매각사측 원인서류 스캔본"
path_out = path
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
addInfo = "서울 채권별 25.퀀텀 5.매각사 원인 (img2pdf) 10802102~"
add_file_name = " 기타 매각사자료모음" # 앞공백 추가
############################################
path_success_log = r'D:\0.전산\1.진행중과업\파일서버개편\구스캔폴더 작업log\success'
path_fail_log = path
dir_list = [d.name for d in os.scandir(path) if d.is_dir()] # 에러폴더 만들기 전에 
total=len(dir_list)

success, fail, notImg, onePage, = [], [], [], []
empthFolder = 0

for dir in tqdm(dir_list, total=total) :
    if int(dir) >= 10802102 :
        image = None # 폴더별로 초기화
        path_img_list = []
        file_list = [x.name for x in os.scandir(join(path, dir)) if x.is_file()]
        for f in file_list :
            if p_extension.search(f) :
                ## 짝수페이지 버리기 #######
                # num = int(re.search(r"(?<=_)\d{4,4}", f).group())
                # if num % 2 != 0 :
                ###########################
                path_img_list.append(join(path, dir, f))
            else :
                notImg.append([path, dir, f])

        images = []
        if len(path_img_list) > 0 :
            for path_image in path_img_list :
                try :
                    image = Image.open(join(path_image))
                except Exception as e:
                    fail.append([path_image, e.__class__, "open 에러 확장자 변경해봐"])
                    continue # 손상 파일 제외(손상파일 맞는지 수동 확인)

                for i, page in enumerate(ImageSequence.Iterator(image)):
                    try : 
                        # page.mode != "RGB" 넣으면 images.append를 어디에 놓던 이상하게 에러 난다.(페이지 반복)
                        page = page.convert("RGB") 
                        images.append(page)
                    except Exception as e:
                        fail.append([path_image, e.__class__, "rgb convert에러"])
                        continue

            if len(images) > 0 :
                # 파일명 ###############################
                new_stem = dir + add_file_name
                new_name = new_stem + ".pdf"
                output = join(path_out, new_name) # 저장파일 전체경로
                sn = 0
                while os.path.exists(output) :
                    sn += 1
                    new_name = new_stem + " (" + str(sn) + ")"+ ".pdf"
                    output = join(path_out, new_name)
                try :
                    if len(images) == 1:
                        images[0].save(output)
                    else:
                        images[0].save(output, save_all=True, append_images=images[1:])
                    success.append([path, dir, output])
                except Exception as e : 
                    fail.append([path, dir, e.__class__, "저장 에러"])
        else : empthFolder += 1


loss = total - len(success) -len(fail) - empthFolder # 폴더 수 - PDF만든거 - pdf못 만든거
print(loss, "개 누수")
print(f"전체 : {total}개, 파일변환 : {len(success)}개, 실패{len(fail)}개, 빈폴더{empthFolder}개")
print(f"{len(notImg)}개 이미지 아닌 파일 있음")

write_log_csv_upload(success, path_success_log, addInfo)
write_log_csv_upload(fail, path_fail_log, "파일병합 실패" + addInfo)
        

100%|██████████| 472/472 [14:03<00:00,  1.79s/it] 

117 개 누수
전체 : 472개, 파일변환 : 244개, 실패111개, 빈폴더0개
0개 이미지 아닌 파일 있음


In [7]:
print(dir, f, path_image)

10802101 LA17120236_2015000181LA17120236001.tif D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\25.퀀텀_솔림헬프\5.매각사측 수령 스캔본\매각사측 원인서류 스캔본\10802101\LA17120236_100600019120171212남정명(동경야시장).tif


In [8]:
write_log_csv_upload(success, path_success_log, addInfo+"~10802100")
write_log_csv_upload(fail, path_fail_log, "파일병합 실패~10802100")

##### 특정 파일명만 골라 작업하기

In [22]:
# 오류 없다는 가정으로 로그 안 만들었음... 주의!!
###########################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\2.원인서류(대출신청서)\1. 원인서류 전체 (파일명 변경완료)\에러"
p = re.compile("원인서류") # match
###########################################
for root, dirs, files in os.walk(path) :
    for f in files :
        if p.match("원인서류", f) :
            stem, ext = os.path.splitext(f)
            images = []
            image = Image.open(join(root, f))   # 1. 이미지 읽기
            for i, page in enumerate(ImageSequence.Iterator(image)): 
                page = page.convert("RGB") # 2. 이미지 convert
                images.append(page) # 3. convert된 이미지를 리스트에 저장

            new_stem = os.path.split(root)[-1]
            output = join(path, new_stem + ".pdf")
            if len(images) == 1: # 4. images[0]을 저장. save_all과 append_images 속성으로 다음페이지 붙여 저장
                images[0].save(output)
            else:
                images[0].save(output, save_all=True,append_images=images[1:]) # pdf_path 는 마지막 tiff파일명으로 대체

            image.close()
            os.rename(join(root, f), join(path, "원인서류 원본", new_stem + ext))

##### 통합 완료한 img 파일 지우기/옮기기

In [2]:
# 지우기
############################################
path = r"D:\구 스캔파일\광주 공유폴더\모음\대출원장\티와이머니_2 원인서류 이미지"
path_dst_base = r"D:\업로드완료\광주 모음 대출원장 티와이_2원인 img파일"
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
addInfo = "광주 모음 대출원장 티와이_2원인 img삭제"
#add_file_name = "A"
############################################
path_success_log = r'D:\0.전산\1.진행중과업\파일서버개편\구스캔폴더 작업log\success'
path_fail_log = path
success, fail = [], []

dir_list = [d.name for d in os.scandir(path) if d.is_dir()]
total=len(dir_list)
pdf_count = 0
for d in tqdm(dir_list, total=total) :
    path_img_list = []
    file_list = [x.name for x in os.scandir(join(path, d)) if x.is_file()]
    add_pdf_count = False
    for f in file_list :
        if p_extension.search(f) :
            add_pdf_count = True
            try :
                # os.remove(join(path, d, f))
                dst_dir = join(path_dst_base, d, "B") #################
                if not os.path.exists(dst_dir) :
                    os.makedirs(dst_dir)
                os.rename(join(path, d, f), join(dst_dir, f))
                success.append([path, d, f])
            except Exception as e :
                fail.append([path, d, f, e.__class__])   ################### 에러 있는 폴더를 에러 폴더로 모으는 코드 작성
                pass
    if add_pdf_count :
        pdf_count += 1

print(f"{len(success)}개 이미지 파일 삭제, {len(fail)}개 이미지 파일 삭제 실패")
print(f"pdf파일은 총 {pdf_count}개여야 함")

write_log_csv_upload(success, path_success_log, addInfo)
write_log_csv_upload(fail, path_fail_log, "img파일 삭제 실패")

100%|██████████| 928/928 [18:47<00:00,  1.22s/it] 


151237개 이미지 파일 삭제, 0개 이미지 파일 삭제 실패
pdf파일은 총 902개여야 함
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\모음\대출원장\티와이머니_2 원인서류 이미지


### 단일 폴더

##### 여러 img파일을 하나의 pdf로

In [16]:
# 폴더내 이미지 파일 하나로 합치기. path에 생성
############################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\2.원인서류(대출신청서)"
add_file_name = path.split(os.path.sep)[-1] + " 원인서류 여타매각사자료포함"
############################################
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 

img_list = [join(path, f) for f in os.listdir(path) if p_extension.search(f)]

output = join(path, "20466036 이화준 기타 매각사자료"+ ".pdf")
if os.path.exists(output) :
    sn = 0
    while os.path.exists(output) :
        sn += 1
        output = join(path, add_file_name + str(sn) + ".pdf")

images = []
for path_image in img_list :
    
    image = Image.open(path_image)   # 1. 이미지 읽기
    
    for i, page in enumerate(ImageSequence.Iterator(image)): 
        page = page.convert("RGB") # 2. 이미지 convert
        images.append(page) # 3. convert된 이미지를 리스트에 저장

if len(images) == 1: # 4. images[0]을 저장. save_all과 append_images 속성으로 다음페이지 붙여 저장
    images[0].save(output)
else:
    images[0].save(output, save_all=True,append_images=images[1:]) # pdf_path 는 마지막 tiff파일명으로 대체

##### 여러 img파일을 각각 pdf로

In [3]:
############################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\2.원인서류(대출신청서)"
addInfo = "서울 채권별 20.우담 2.원인 리드~한투 (img2pdf)"
############################################
success, fail = [], []
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
file_list = [f for f in os.listdir(path) if p_extension.search(f)]
err_dir = join(path, "에러")
output_dir = join(path, "pdf번환")
if not os.path.exists(err_dir) : os.mkdir(err_dir)
if not os.path.exists(output_dir) : os.mkdir(output_dir)

for f in tqdm(file_list, total=len(file_list)):
    try :
        stem, ext = os.path.splitext(f)
        images = []
        image = Image.open(join(path, f))   

        for i, page in enumerate(ImageSequence.Iterator(image)): 
            page = page.convert("RGB") 
            images.append(page) 

        output = join(output_dir, stem + ".pdf")
        if len(images) == 1: 
            images[0].save(output)
        else:
            images[0].save(output, save_all=True, append_images=images[1:]) 
        success.append([path, f, output])
        image.close()
    except Exception as e :
        image.close()
        os.rename(join(path, f), join(err_dir, f))
        fail.append([path, f, err_dir, f, e.__class__, e])

write_log_csv_upload(success, path_success_log, addInfo)
write_log_csv_upload(fail, path, "에러")        

100%|██████████| 59/59 [00:09<00:00,  6.32it/s]


NameError: name 'addInfo' is not defined

### pdf파일 페이지 쪼개기

##### 모든 페이지 쪼개기

In [ ]:
# 단일페이지
path = r"D:\구 스캔파일\광주 공유폴더\통합파일\(6.케이에스자산관리대부_헬프)\3.채권양도통지서\펜타양통"
file = "이덕윤 (망)노희순의 상속인) 550607.pdf"
pdfReader = PdfFileReader(join(path, file), "rb")

# 페이지 하나씩 받아와서 저장하는 반복문
for pageNum in range(pdfReader.numPages):
    
    # step3.새로 만들 pdf 객체 생성 (계속 누적되지 않기 위해 for문 안으로 넣음)
    pdfWriter = PdfFileWriter()
    
    # step4.기존 PDF에서 한 페이지씩 가져오기
    page = pdfReader.getPage(pageNum)
    
    # step5.위에서 가져온 페이지를 새로 만든 PDF에 붙여넣기
    pdfWriter.addPage(page)

    # step6.새로운 PDF 파일을 해당 경로('./')에 원하는 이름으로 저장
    # (이름을 계속 다르게 해주기 위해서 f 문자열 포매팅 개념을 이용)
    pdfWriter.write(open(join(path, f"분할한 PDF 파일 {pageNum+1}.pdf"), "wb"))

In [2]:
# 폴더내 모든 파일
# 파일명 순서에 유의할 것 0으로 자릿수 채우면 정렬 잘됨(stem.zfill(자릿수))
##########################################
path = r"D:\구 스캔파일\서울\광주채권\외장기본있던것\★공유업데이트 2021.0712\■양도통지서 스캔\한울(전체)양도통지서(헬프매각)\세종공평양통\세종양도통지서(1차)"
addInfo = "서울 광주채권 외장 공유업데이트 양통스캔 한울(전체) 세종 1차"
num = 1 #시작번호 그대로
##########################################
cnt = 0
file_list = [x.name for x in os.scandir(path) if x.is_file() and re.search("pdf|PDF", x.name)]
total = len(file_list)
#file_list
dst_dir = join(path, "분할") # file_list 밑에 나와야
if not os.path.exists(dst_dir) : os.mkdir(dst_dir)

for f in tqdm(file_list, total=total) :
    source_file = PdfFileReader(join(path, f), "rb")

    for pageNum in range(source_file.numPages):
        # 페이지마다 새로운 파일이니 for문 안에
        split_file = PdfFileWriter()
        page = source_file.getPage(pageNum)
        split_file.addPage(page)

        split_file.write(open(join(dst_dir, str(num) + ".pdf"), "wb"))
        cnt += 1
        num += 1

print(f"{total}개의 파일을 {cnt}개의 파일로 분리 성공")

100%|██████████| 11/11 [00:09<00:00,  1.18it/s]

11개의 파일을 1101개의 파일로 분리 성공


In [13]:
file_list

['동백,새마을1(1~73) (2).pdf',
 '동백,새마을2(74~153) (2).pdf',
 '동백,새마을3(154~226) (2).pdf',
 '동백,새마을4(227~314) (2).pdf']

In [8]:
num = 6012622
if ((num == 6012622) or (num == 6012627) or (num == 6012972)) :
    print(num)
    num += 1
print(num)

6012622
6012623


##### 기준페이지에서 둘로 나누기

In [ ]:
#######################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\2.원인서류(대출신청서)\양통분리"
addInfo = "서울 채권별 20.우담 2.원인 리드~한투(원인양통분리)"
p = re.compile("(\d+)-(\d+)")
#######################################
file_list = [x.name for x in os.scandir(path) if x.is_file() and re.search("pdf|PDF", x.name)]
total = len(file_list)
# 분리할 파일 #######################################
base_dir = join(path, "기타") # file_list 밑에 나와야
split_dir = join(path, "원인서류")
if not os.path.exists(base_dir) : os.mkdir(base_dir)
if not os.path.exists(split_dir) : os.mkdir(split_dir)
# ########## #######################################
success, fail = [], []
path_fail_log = path


for f in tqdm(file_list, total=total) : 
    try :
        ##################### 
        source_file = PdfFileReader(join(path, f), "rb")
        base_file_name = re.sub("원인서류", "기타 매각사자료", f) # 기타
        split_file_name = f

        if p.search(f) :
            s_num = int(p.search(f).group(1)) - 1
            e_num = int(p.search(f).group(2)) - 1
        # else : s_num = 1
        # total_pages = source_file.numPages
        # num_split = total_pages - s_num
        else :
            pass
        
        base_file = PdfFileWriter()
        split_file = PdfFileWriter()

        # 양통 추출(0~)
        for n in range(source_file.numPages):
            page = source_file.getPage(n)
            if (n < s_num) or (n > e_num):
                base_file.addPage(page)
            else :
                split_file.addPage(page)
            
        base_file.write(open(join(base_dir, base_file_name), "wb"))
        split_file.write(open(join(split_dir, split_file_name), "wb"))
        success.append([path, f, base_file_name, split_file_name])
    except Exception as e :
        fail.append([path, f, e.__class__, e])
        continue

print(f"{total}개 중 {len(success)}개 분리성공, {len(fail)}개 분리실패")
write_log_csv_upload(success, path_success_log, addInfo)
write_log_csv_upload(fail, path, "fail")

##### 페이지 범위로 분리하기 

In [6]:
#######################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\2.원인서류(대출신청서)\양통분리"
addInfo = "서울 채권별 20.우담 2.원인 리드~한투(원인양통분리)"
p = re.compile("양통(\d+)")
#######################################
file_list = [x.name for x in os.scandir(path) if x.is_file() and re.search("pdf|PDF", x.name)]
total = len(file_list)
# 분리할 파일 #######################################
base_dir = join(path, "원인서류") # file_list 밑에 나와야
split_dir = join(path, "양통")
if not os.path.exists(base_dir) : os.mkdir(base_dir)
if not os.path.exists(split_dir) : os.mkdir(split_dir)
# ########## #######################################
success, fail = [], []
path_fail_log = path


for f in tqdm(file_list, total=total) : 
    try :
        ##################### 
        source_file = PdfFileReader(join(path, f), "rb")
        base_file_name = re.sub("양통\d+", "", f) # 기타
        split_file_name = re.sub("원인서류", "매입전", f) 

        if p.search(f) :
            criterion_num = int(p.search(f).group(1))
        else : criterion_num = 1
        total_pages = source_file.numPages
        num_split = total_pages - criterion_num
        
        base_file = PdfFileWriter()
        split_file = PdfFileWriter()

        # 양통 추출(0~)
        for n in range(total_pages):
            page = source_file.getPage(n)
            if n < num_split :
                base_file.addPage(page)
            else :
                split_file.addPage(page)
            
        base_file.write(open(join(base_dir, base_file_name), "wb"))
        split_file.write(open(join(split_dir, split_file_name), "wb"))
        success.append([path, f, base_file_name, split_file_name])
    except Exception as e :
        fail.append([path, f, e.__class__, e])
        continue

print(f"{total}개 중 {len(success)}개 분리성공, {len(fail)}개 분리실패")
write_log_csv_upload(success, path_success_log, addInfo)
write_log_csv_upload(fail, path, "fail")

  0%|          | 0/11 [00:00<?, ?it/s]Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
 55%|█████▍    | 6/11 [00:00<00:00, 40.41it/s]Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
100%|██████████| 11/11 [00:00<00:00, 38.82it/s]

11개 중 11개 분리성공, 0개 분리실패
log 파일이 비어있습니다. path : D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\2.원인서류(대출신청서)\양통분리


### pdf파일 회전하기

##### 일반

In [5]:
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\24.스마트저축_솔림헬프\1.스마트에서 파일로 준 원인서류\초본\새 폴더"
path_out = join(path, "회전")
rotage_angle = 180

if not os.path.exists(path_out) :
    os.mkdir(path_out)

p_extension = re.compile('pdf$', re.I) 
file_list = [f.name for f in os.scandir(path) if p_extension.search(f.name)]
total = len(file_list)

for f in tqdm(file_list, total=total) : 
    pdfReader = PdfFileReader(join(path, f), "rb")
    rotated_file = PdfFileWriter()
    total_pages = pdfReader.numPages

    # 반시계방향
    for n in range(total_pages) :
        page = pdfReader.getPage(n)
        page.rotateCounterClockwise(rotage_angle)
        rotated_file.addPage(page)

    rotated_file.write(open(join(path_out, f), "wb"))

100%|██████████| 1/1 [00:00<00:00, 62.71it/s]


##### 일부페이지(전부도 포함)

In [5]:
#######################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\2.원인서류(대출신청서)\상하"
addInfo = "서울 채권별 20.우담 2.원인 리드~한투(회전)"
path_out = join(path, "회전")
p = re.compile("상하(\d+)")
rotage_angle = 180
#######################################
if not os.path.exists(path_out) : os.mkdir(path_out)
success, fail = [], []

p_extension = re.compile('pdf|PDF', re.I) 

file_list = [f.name for f in os.scandir(path) if p_extension.search(f.name)]
total = len(file_list)

for f in tqdm(file_list, total=total) : 
    try :
        source_file = PdfFileReader(join(path, f), "rb")
        rotated_file = PdfFileWriter()
        total_pages = source_file.numPages # 실제쪽수 1 ~

        if p.search(f) :
            criterion_num = int(p.search(f).group(1))
        else : # 전체회전
            criterion_num = total_pages

        for n in range(total_pages): # 0 ~
            page = source_file.getPage(n)
            if n < (total_pages-criterion_num) : 
                pass 
            else : 
                page.rotateCounterClockwise(rotage_angle)
            rotated_file.addPage(page)
        
        rotated_file.write(open(join(path_out, f), "wb"))
        success.append([path, f, total_pages, criterion_num, "전체/회전페이지"])
    except Exception as e :
        fail.append([path, f, e.__class__, e])

print(f"{total}개 중 {len(success)}개 회전성공, {len(fail)}개 회전실패")
write_log_csv_upload(success, path_success_log, addInfo)
write_log_csv_upload(fail, path, "fail")

100%|██████████| 45/45 [00:01<00:00, 38.22it/s]

45개 중 45개 회전성공, 0개 회전실패
log 파일이 비어있습니다. path : D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\2.원인서류(대출신청서)\상하


In [15]:
import shutil
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\2.원인서류(대출신청서)\에러파일\pdf로 바꾸면 열릴 파일"

for f in os.listdir(path) :
    if os.path.isfile(join(path, f)) :
        stem, ext = os.path.splitext(f)
        shutil.copy2(join(path, f), join(path, "pdf", stem + ".pdf"))

### pdf 파일 텍스트 읽기 

In [1]:
############################# 텍스트 읽기 테스트
from PyPDF2 import PdfFileReader, PdfFileWriter
# step2.기존 PDF 불러오기
path = r"D:\구 스캔파일\광주 공유폴더\통합파일\(7.KB국민카드_솔림헬프)\3.채권양도통지서\국민카드 양통\광주2-4차 20통.pdf"

pdfReader = PdfFileReader(path, "rb")
page = pdfReader.getPage(0)
page.extract_text()



' 채 권 양 도 통 지 서  \n수신인  :  이병창  \n        05039  서울 광진구 자양로22길 9-5, (구의동,1층)  \n \n발신인 : 주식회사 솔림헬프자산관리대부  \n         04790  서울시 성동구 성수일로77, 801호(서울숲 IT밸리)  \n \n본 통지서는 주식회사 KB국민카드[이하 “양도인”]와 양수인 주식회사 솔림헬프자산관리대부[이하 “양수인”]와의  \n2021년 04월 21일자 자산양수도 계약에 따라 체결된 채권매매계약과 관련됩니다. \n 양도인은 위 채권매매계약에 따라 2021년 04월 30일(“양도일”)자로 양수인에게 아래에 명시된 채권 및 이와 관련된 \n모든 원금, 이자, 지연손해금 기타 이와 관련된 권리 일체(담보 혹은 보증인 있는 경우  이를 포함, 법적절차 및 기타 \n모든권리)를 양도하였음을 알려드립니다.   \n아울러 위 채권양도와 관련한 권리의 이전과 함께 주식회사 KB국민카드는 「신용정보의 이용 및 보호에 관한 \n법률」제32조 7항 및 동법 시행령에 의거 채무자의 개인신용정보가  양수인인 주식회사 솔림헬프자산관리대부 (연락처 : \n062-384-2255)에게 이전·제공하였음을 통지합니다. \n \n또한, 양수인은 양도대상 채권에 대하여 신용정보 이용 및 보호에 관한 법률 및 신용정보회사간 신용정보관리 규약에 의거 신용정보전산망에 등록할 수 있음을 통지하며, 향후 귀하의 채무변제 및 기타 상담은 아래 양수인 \n연락처로 문의하여 주시기 바랍니다. \n \n※ 본 고지는 채무자가 신고한 최종 주소로 안내함에 따라 해당 정보를 갱신하지 않아 발생하는 불이익은 채무자에게 있습니다. \n \n* 양도채권명세 (기준일 : 2021년 04월 1일)                                                            (단위: 원) \n양도기관 성함/주민번호 일련번호 미상환원금잔액 이자 가지급금 합계 \n주식회사 \nKB국민카드 이병창 \n \n600815-\n1**

### img2pdf - 에러나고 느리고 출력도 이상하니 쓰지마

In [7]:
import img2pdf
### 여러장 짜리 이것도 됨
############################################
path = r"C:\Users\SL\Desktop\test"
path_out = path
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
add_file_name = ""
############################################
path_success_log = r'D:\0.전산\1.진행중과업\파일서버개편\구스캔폴더 작업log\success'
path_fail_log = path

success = []
fail = []
notImg = []

dir_list = [d.name for d in os.scandir(path) if d.is_dir()]
total=len(dir_list)

for d in tqdm(dir_list, total=total) :
    if int(d[:8]) > 0 :
        temp = []
        file_list = os.listdir(join(path, d))
        for f in file_list :
            if p_extension.search(f) :
                ########################### 짝수페이지 버리기
                # num = int(re.search(r"(?<=_)\d{4,4}", f).group())
                # if num % 2 != 0 :
                ###########################
                temp.append(join(path, d, f)) # 전체경로를 저장해야 함
                # else : continue
            else :
                notImg.append([path, d, f])
                continue
        
        if len(temp)>0 :
            try :
                output = join(path_out, d + add_file_name + ".pdf")
                if os.path.exists(output) :
                    sn = 0
                    while os.path.exists(output) :
                        sn += 1
                        output = join(path_out, d + add_file_name + str(sn) + ".pdf")

                with open(output,"wb") as f:
                    f.write(img2pdf.convert(temp))

                success.append([d, output])
            except Exception as e :
                fail.append([path, d, e.__class__])

loss = total - len(success) -len(fail) # 폴더 수 - PDF만든거 - pdf못 만든거
print(loss, "개 누수")
print(f"전체 : {total}개, 파일변환 : {len(success)}개, 실패{len(fail)}개")
print(f"{len(notImg)}개 이미지 아닌 파일 있음")

# write_log_csv_upload(success, path_success_log, "파일병합 성공")
# write_log_csv_upload(fail, path_fail_log, "파일병합 실패")

100%|██████████| 1/1 [00:03<00:00,  3.98s/it]

0 개 누수
전체 : 1개, 파일변환 : 1개, 실패0개
0개 이미지 아닌 파일 있음


In [37]:
cnt = 0
for root, dirs, files in os.walk(path) :
    for dir in dirs :
        try :
            os.rmdir(join(root, dir))
            cnt += 1
        except :
            pass
print(cnt)

27


In [19]:
print(len(success), len(fail))

0 7


In [7]:
print(*notImg, sep="\n")

['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498815 이갑선 원인서류', '0220141270521_신용대출신청서_0001_61923136.pdf']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498815 이갑선 원인서류', '0220161031700_신용대출신청서_0001_66553331.pdf']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498832 전현철 원인서류', '전현철_06775710058933_.pdf']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498841 임수영 원인서류', '049783386720004_임수영.pdf']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498841 임수영 원인서류', '4101090007240295_임수영.pdf']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20498868 김명식 원인서류', '10055793106-1.pdf']
['D:\\extraKe

In [41]:
# 이미지 파일 아닌 거 처리하기
new = notImg[127:]

In [10]:
for x in notImg : 
    if not re.search("pdf|PDF", x[2]) :
        print(x)

['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20499526 김광현 원인서류', '19031100023_녹취파일.wav']
['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류', '20500043 고영세 원인서류', '4347014100036.wav']


In [15]:
dst_dir = r"D:\extraKey\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\22.베리타스_솔림헬프\원인서류 notImg"
success = []
fail = []
for x in notImg :

    src = join(x[0], x[1], x[2])
    ext = os.path.splitext(x[2])[1]
    new_name = x[1] + add_file_name + ext

    sn = 0
    while os.path.exists(join(dst_dir, new_name)) :
        sn += 1
        new_name = x[1] + add_file_name +" " + str(sn) + ext
    try :
        os.rename(src, join(dst_dir, new_name))
        success.append([src, join(dst_dir, new_name)])
    except Exception as e:
        fail.append([src, e.__class__, e])

print(f"이동 {len(success)}개, 실패{len(fail)}개")

이동 242개, 실패1개


In [16]:
print(fail)

[['D:\\extraKey\\서울\\! 채권별 스캔본 (한울가람, 9001~)\\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\\22.베리타스_솔림헬프\\1.베리타스에서 파일로 준 원인서류\\20498815 이갑선 원인서류\\0220141270521_신용대출신청서_0001_61923136.pdf', <class 'FileNotFoundError'>, FileNotFoundError(2, '지정된 파일을 찾을 수 없습니다')]]


In [30]:
f = "L41805056190001_0001.jpg"
m = re.search(r"(?<=_)\d{4,4}", f).group()
m

'0001'

### img2pdf 단일 폴더 내 파일 합치기

In [38]:
# # 폴더내 이미지 파일 하나로 합치기. path에 생성
# ############################################
# path = r"D:\구 스캔파일\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류\20428545"
# ############################################
# p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
# add_file_name = path.split(os.path.sep)[-1] + " 원인서류 여타매각사자료포함"

# img_list = [join(path, f) for f in os.listdir(path) if p_extension.search(f)]
# output = join(path, add_file_name + ".pdf")
# if os.path.exists(output) :
#     sn = 0
#     while os.path.exists(output) :
#         sn += 1
#         output = join(path, add_file_name + str(sn) + ".pdf")

# with open(output,"wb") as f:
#     f.write(img2pdf.convert(img_list))

KeyError: 262